https://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html

In [7]:
from math import sin

import pickle
import random
import numpy

from deap import creator, base, tools, algorithms

In [4]:
def evalFct(individual):
    x = individual[0]
    return (x-5)**2 * sin(x) * (x/3)

def feasible(individual):
    if 3 < individual[0] < 7:
        return True
    return False

def distance(individual):
    return (individual[0] - 5.0)**2

toolbox = base.Toolbox()
toolbox.register("evaluate", evalFct)
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 7.0, distance))



In [8]:
IND_SIZE = 1
NGEN = 1000
FREQ = 100
CXPB, MUTPB = 0.5, 0.2

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)


In [10]:
# main algo

population = toolbox.population(n=300)
start_gen = 0
halloffame = tools.HallOfFame(maxsize=1)
logbook = tools.Logbook()

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register('max', numpy.max)

for gen in range(start_gen, NGEN):
    population = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)

    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesss = toolbox.map(toolbox.evaluate, population)
    for ind, fit in zip(invalid_ind, fitnesss):
        ind.fitness.values = fit

    halloffame.update(population)
    record = stats.compile(population)
    logbook.record(gen=gen, evals=len(invalid_ind), **record)

    population = toolbox.select(population, k=len(population))

    if gen % FREQ == 0:
        cp = dict(population=population, generation=gen, halloffame=halloffame, 
                 logbook=logbook, rndstate=random.getstate())

        with open('checkpoint_name.pkl', 'wb') as cp_file:
            pickle.dump(cp, cp_file)


print(logbook)

ValueError: empty range for randrange() (1,1, 0)